In [142]:
import numpy as np
import numexpr as ne
import math
import dataclasses as dc

def create_concrete_combos(length, step, depth):
    # Create array A
    A = np.arange(length, length + 1.05, 0.05)
    
    # Create array B
    B = A + step
    
    # Create array C
    C = np.arange(depth, (2*length + step)/4, 0.05)
    
    # Create all combinations using numpy's meshgrid
    A_mesh, B_mesh, C_mesh = np.meshgrid(A, B, C, indexing='ij')
    combinations = np.stack([A_mesh.ravel(), B_mesh.ravel(), C_mesh.ravel()], axis=-1)
    
    # Calculate A*B*C using numexpr
    A_vals = combinations[:, 0]
    B_vals = combinations[:, 1]
    C_vals = combinations[:, 2]
    product_ABC = ne.evaluate('A_vals * B_vals * C_vals')
    
    # Sort combinations based on A*B*C
    sorted_indices = np.argsort(product_ABC)
    sorted_combinations = combinations[sorted_indices]
    
    return sorted_combinations

def create_steel_combos():
    # Create array A
    A = np.array([0.012,0.016,0.020,0.024,0.028,0.032,0.036,0.040])
    
    # Create array B
    B = np.array([0.1, 0.120, 0.125, 0.14, 0.15, 0.16, 0.175, 0.18, 0.2, 0.22, 0.225, 0.24, 0.25, 0.26, 0.275, 0.28, 0.3])
    

    # Create all combinations using numpy's meshgrid
    A_mesh1, B_mesh1, A_mesh2, B_mesh2 = np.meshgrid(A, B, A, B, indexing='ij')
    combinations = np.stack([A_mesh1.ravel(), B_mesh1.ravel(), A_mesh2.ravel(), B_mesh2.ravel()], axis=-1)
    
    # Calculate A*B*A*B using numexpr
    A_vals1 = combinations[:, 0]
    B_vals1 = combinations[:, 1]
    A_vals2 = combinations[:, 2]
    B_vals2 = combinations[:, 3]
    product_ABAB = ne.evaluate('A_vals1 * B_vals1 * A_vals2 * B_vals2')
    
    # Sort combinations based on A*B*A*B
    sorted_indices = np.argsort(product_ABAB)
    sorted_combinations = combinations[sorted_indices]
    
    return sorted_combinations

def concrete_filter(conc_arr):
    # Extract individual columns
    A = conc_arr[:, 0]
    B = conc_arr[:, 1]
    C = conc_arr[:, 2]
    
    # Create a boolean mask for the condition using numexpr
    mask = ne.evaluate('(0 < (8000 + 1500 + 6 * A * B * C) / (250 * A * B)) & ((8000 + 1500 + 6 * A * B * C) / (250 * A * B) < 1)')
    
    # Apply the mask to filter the array
    return conc_arr[mask]

def steel_filter(conc_arr, steel_arr):
    # Get the smallest value in the last column of array1
    depth = np.min(conc_arr[:, -1])
    
    # Prepare the dictionary for numexpr
    context = {
        'barL': steel_arr[:, 0],
        'ctsL': steel_arr[:, 1],
        'barW': steel_arr[:, 2],
        'ctsW': steel_arr[:, 3],
        'D': depth,
        'pi': math.pi,
        'sqrt_25': math.sqrt(25)
    }

    # Calculate the conditions using numexpr
    condition1 = ne.evaluate('31250 * barL**2 * (-barL - 2*0.05 + 2*D) * pi / (57 * ctsL * D**2 * sqrt_25) > 1', context)
    condition2 = ne.evaluate('31250 * barW**2 * (-barW - 2*0.05 + 2*D) * pi / (57 * ctsW * D**2 * sqrt_25) > 1', context)

    # Combine conditions using logical AND
    mask = np.logical_and(condition1, condition2)
    
    # Filter array2 based on the mask
    filtered_rows = steel_arr[mask]
    
    return filtered_rows  # Return the filtered array2

# Example usage:
X, Y, Z = 6.1, -0.3, 2

# Run the function and print the result size
conc = create_concrete_combos(X, Y, Z) 
conc = concrete_filter(conc)
reo = create_steel_combos()
reo = steel_filter(conc, reo)
print(conc.size)
print(reo.size)

17229
23104


In [143]:
@dc.dataclass
class Foundation:
    L: float = 0.0
    W: float = 0.0
    D: float = 0.0
    Cvr: float = 0.0
    ColL: float = 0.0
    ColW: float = 0.0
    Pdl: float = 0.0
    Pll: float = 0.0
    BP: float = 0.0
    fc: float = 0.0
    barL: float = 0.0
    ctsL: float = 0.0
    barW: float = 0.0
    ctsW: float = 0.0

    def __post_init__(self):
        for field in dc.fields(self):
            setattr(self, field.name, np.atleast_1d(getattr(self, field.name)))

def calculate_SWt(fdn):
    return 6 * fdn.D * fdn.L * fdn.W

def calculate_Pult(fdn):
    return 1.2 * (fdn.Pdl + calculate_SWt(fdn)) + 1.5 * fdn.Pll

def calculate_BPmax(fdn):
    return (fdn.Pdl + calculate_SWt(fdn) + fdn.Pll) / (fdn.L * fdn.W)

def calculate_BPult(fdn):
    return calculate_Pult(fdn) / (fdn.L * fdn.W)

def calculate_AstL(fdn):
    return 250000 / fdn.ctsL * fdn.barL**2 * np.pi

def calculate_AstW(fdn):
    return 250000 / fdn.ctsW * fdn.barW**2 * np.pi

def calculate_dsL(fdn):
    return fdn.D - fdn.Cvr - fdn.barL / 2

def calculate_dsW(fdn):
    return fdn.D - fdn.Cvr - fdn.barL - fdn.barW / 2

def calculate_AstminL(fdn):
    return (228 * fdn.D**2 * np.sqrt(fdn.fc)) / calculate_dsL(fdn)

def calculate_AstminW(fdn):
    return (228 * fdn.D**2 * np.sqrt(fdn.fc)) / calculate_dsW(fdn)

def calculate_alpha(fdn):
    return 0.85 - 0.0015 * fdn.fc

def calculate_gamma(fdn):
    return 0.97 - 0.0025 * fdn.fc

def calculate_MultL(fdn):
    return ((7 * fdn.ColL - 10 * fdn.L) ** 2 * (-9 * calculate_SWt(fdn) + 10 * calculate_BPult(fdn) * fdn.L * fdn.W)) / (8000 * fdn.L * fdn.W)

def calculate_MultW(fdn):
    return ((7 * fdn.ColW - 10 * fdn.W) ** 2 * (-9 * calculate_SWt(fdn) + 10 * calculate_BPult(fdn) * fdn.L * fdn.W)) / (8000 * fdn.L * fdn.W)

def calculate_AstshrL(fdn):
    return 100 * calculate_MultL(fdn) / (calculate_dsL(fdn)*(50-9*calculate_gamma(fdn)))

def calculate_AstshrW(fdn):
    return 100 * calculate_MultW(fdn) / (calculate_dsW(fdn)*(50-9*calculate_gamma(fdn)))

def calculate_AstreqL(fdn):
    return np.maximum(calculate_AstminL(fdn), calculate_AstshrL(fdn))

def calculate_AstreqW(fdn):
    return np.maximum(calculate_AstminW(fdn), calculate_AstshrW(fdn))

def calculate_kuL(fdn):
    return calculate_AstreqL(fdn) / (2000 * calculate_alpha(fdn) * calculate_dsL(fdn) * fdn.fc * calculate_gamma(fdn) * fdn.L)

def calculate_kuW(fdn):
    return calculate_AstreqW(fdn) / (2000 * calculate_alpha(fdn) * calculate_dsW(fdn) * fdn.fc * calculate_gamma(fdn) * fdn.W)

def calculate_phiL(fdn):
    return np.minimum(0.85, np.maximum(0.65, 1.24 - 13 * calculate_kuL(fdn) / 12))

def calculate_phiW(fdn):
    return np.minimum(0.85, np.maximum(0.65, 1.24 - 13 * calculate_kuW(fdn) / 12))

def calculate_fMuoL(fdn):
    return (calculate_AstL(fdn) * calculate_dsL(fdn) * calculate_phiL(fdn)) / 2 - (calculate_AstL(fdn)**2 * calculate_phiL(fdn)) / (8000 * calculate_alpha(fdn) * fdn.fc)

def calculate_fMuoW(fdn):
    return (calculate_AstW(fdn) * calculate_dsW(fdn) * calculate_phiW(fdn)) / 2 - (calculate_AstW(fdn)**2 * calculate_phiW(fdn)) / (8000 * calculate_alpha(fdn) * fdn.fc)

def calculate_CLR(fdn):
    return calculate_BPult(fdn) * (fdn.ColW + calculate_dsL(fdn)) * (fdn.ColL + calculate_dsL(fdn))

def calculate_VPult(fdn):
    return calculate_Pult(fdn) - calculate_CLR(fdn)

def calculate_fcv(fdn):
    return 0.17 * (1 + 2 / np.maximum(fdn.ColL / fdn.ColW, fdn.ColW / fdn.ColL, np.full(fdn.L.shape, 2.0))) * np.sqrt(fdn.fc)

def calculate_fVP(fdn):
    return 1400 * calculate_dsW(fdn) * (fdn.ColL + fdn.ColW + 2 * calculate_dsW(fdn)) * calculate_fcv(fdn)

def calculate_dvL(fdn):
    return np.maximum(0.9 * calculate_dsL(fdn), 0.72 * fdn.D)

def calculate_dvW(fdn):
    return np.maximum(0.9 * calculate_dsW(fdn), 0.72 * fdn.D)

def calculate_VOultL(fdn):
    return 0.5 * (-fdn.ColL - 2 * calculate_dvW(fdn) + fdn.L) * (calculate_BPult(fdn) - (9 * calculate_SWt(fdn)) / (10 * fdn.L * fdn.W))

def calculate_VOultW(fdn):
    return 0.5 * (calculate_BPult(fdn) - (9 * calculate_SWt(fdn)) / (10 * fdn.L * fdn.W)) * (-fdn.ColW - 2 * calculate_dvW(fdn) + fdn.W)

def calculate_MOultL(fdn):
    return ((fdn.ColL + 2 * calculate_dvW(fdn) - fdn.L) ** 2 * (-9 * calculate_SWt(fdn) + 10 * calculate_BPult(fdn) * fdn.L * fdn.W)) / (80 * fdn.L * fdn.W)

def calculate_MOultW(fdn):
    return ((fdn.ColW + 2 * calculate_dvW(fdn) - fdn.W) ** 2 * (-9 * calculate_SWt(fdn) + 10 * calculate_BPult(fdn) * fdn.L * fdn.W)) / (80 * fdn.L * fdn.W)

def calculate_ex1L(fdn):
    return np.minimum((np.maximum(calculate_VOultL(fdn) * calculate_dvL(fdn), calculate_MOultL(fdn)) / calculate_dvL(fdn) + calculate_VOultL(fdn)) / (2 * 200000 * calculate_AstL(fdn)) * 1000, 3 / 1000)

def calculate_ex1W(fdn):
    return np.minimum((np.maximum(calculate_VOultW(fdn) * calculate_dvW(fdn), calculate_MOultW(fdn)) / calculate_dvW(fdn) + calculate_VOultW(fdn)) / (2 * 200000 * calculate_AstW(fdn)) * 1000, 3 / 1000)

def calculate_kvL(fdn):
    return 13 / (25 * (1 + calculate_dvL(fdn)) * (1 + 1500 * calculate_ex1L(fdn)))

def calculate_kvW(fdn):
    return 13 / (25 * (1 + calculate_dvW(fdn)) * (1 + 1500 * calculate_ex1W(fdn)))

def calculate_AngleL(fdn):
    return 29 + 7000 * calculate_ex1L(fdn)

def calculate_AngleW(fdn):
    return 29 + 7000 * calculate_ex1W(fdn)

def calculate_ks(fdn):
    return np.maximum(1 / 2, (10 / 7) * (1 - fdn.D))

def calculate_fVucL(fdn):
    return 700 * calculate_dvL(fdn) * np.sqrt(fdn.fc) * calculate_ks(fdn) * calculate_kvL(fdn)

def calculate_fVucW(fdn):
    return 700 * calculate_dvW(fdn) * np.sqrt(fdn.fc) * calculate_ks(fdn) * calculate_kvW(fdn)

def calculate_Bpr(fdn):
    return calculate_BPmax(fdn) / fdn.BP

def calculate_Mur(fdn):
    return np.maximum(calculate_MultL(fdn) / calculate_fMuoL(fdn), calculate_MultW(fdn) / calculate_fMuoW(fdn))

def calculate_VPr(fdn):
    return calculate_VPult(fdn) / calculate_fVP(fdn)

def calculate_VOr(fdn):
    return np.maximum(calculate_VOultL(fdn) / calculate_fVucL(fdn), calculate_VOultW(fdn) / calculate_fVucW(fdn))

def calculate_Cost(fdn):
    return (calculate_AstW(fdn) / 1000000 * fdn.L + calculate_AstL(fdn) / 1000000 * fdn.W) * 7850 * 3.400 + fdn.L * fdn.W * fdn.D * (130.866 * np.exp(fdn.fc * 0.0111) + 45 + 130) + 2 * fdn.D * fdn.L * fdn.W * 180

def calculate_ReoRatio(fdn):
    return np.maximum(calculate_AstL(fdn) / calculate_AstreqL(fdn), calculate_AstW(fdn) / calculate_AstreqW(fdn))

def calc_D_range(fdn):
    extent = np.max([fdn.L.max(), fdn.W.max()])
    D_start = np.round(extent/6 / 0.05) * 0.05-0.1
    D_end = np.round(extent/2 / 0.05) * 0.05+0.1
    D = np.arange(D_start, D_end + 0.05, 0.05)
    _length = len(D)
    L = np.full(_length, fdn.L)
    W = np.full(_length, fdn.W)
    ColL = np.full(_length, fdn.ColL)
    ColW = np.full(_length, fdn.ColW)
    BP = np.full(_length, fdn.BP)
    fc = np.full(_length, fdn.fc)
    barL = np.full(_length, fdn.barL)
    ctsL = np.full(_length, fdn.ctsL)
    barW = np.full(_length, fdn.barW)
    ctsW = np.full(_length, fdn.ctsW)
    Pdl = np.full(_length, fdn.Pdl)
    Pll = np.full(_length, fdn.Pll)
    Cvr = np.full(_length, fdn.Cvr)
    return ([L, W, D, Cvr, ColL, ColW, Pdl, Pll, BP, fc, barL, ctsL, barW, ctsW])

def calc_min_L(fdn):
    L = (fdn.BP *(fdn.ColL-fdn.ColW)+np.sqrt(fdn.BP*(fdn.BP*(fdn.ColL-fdn.ColW)**2+4 *(fdn.Pdl+fdn.Pll))))/(2* fdn.BP) 
    return np.round(L/0.05)*0.05

def print_foundation_results(fdn):
    print(f"Results for L={fdn.L[0]:.3f}, W={fdn.W[0]:.3f}, D={fdn.D[0]:.3f}, barL={fdn.barL[0]:.3f}, ctsL={fdn.ctsL[0]:.3f}, barW={fdn.barW[0]:.3f}, ctsW={fdn.ctsW[0]:.3f}")
    print(f"SWt: {calculate_SWt(fdn)[0]:.2f}")
    print(f"Pult: {calculate_Pult(fdn)[0]:.2f}")
    print(f"BPmax: {calculate_BPmax(fdn)[0]:.2f}")
    print(f"BPult: {calculate_BPult(fdn)[0]:.2f}")
    print(f"AstL: {calculate_AstL(fdn)[0]:.2f}")
    print(f"AstW: {calculate_AstW(fdn)[0]:.2f}")
    print(f"dsL: {calculate_dsL(fdn)[0]:.2f}")
    print(f"dsW: {calculate_dsW(fdn)[0]:.2f}")
    print(f"AstminL: {calculate_AstminL(fdn)[0]:.2f}")
    print(f"AstminW: {calculate_AstminW(fdn)[0]:.2f}")
    print(f"alpha: {calculate_alpha(fdn)[0]:.2f}")
    print(f"gamma: {calculate_gamma(fdn)[0]:.2f}")
    print(f"MultL: {calculate_MultL(fdn)[0]:.2f}")
    print(f"MultW: {calculate_MultW(fdn)[0]:.2f}")
    print(f"AstshrL: {calculate_AstshrL(fdn)[0]:.2f}")
    print(f"AstshrW: {calculate_AstshrW(fdn)[0]:.2f}")
    print(f"AstreqL: {calculate_AstreqL(fdn)[0]:.2f}")
    print(f"AstreqW: {calculate_AstreqW(fdn)[0]:.2f}")
    print(f"kuL: {calculate_kuL(fdn)[0]:.2f}")
    print(f"kuW: {calculate_kuW(fdn)[0]:.2f}")
    print(f"phiL: {calculate_phiL(fdn)[0]:.2f}")
    print(f"phiW: {calculate_phiW(fdn)[0]:.2f}")
    print(f"fMuoL: {calculate_fMuoL(fdn)[0]:.2f}")
    print(f"fMuoW: {calculate_fMuoW(fdn)[0]:.2f}")
    print(f"CLR: {calculate_CLR(fdn)[0]:.2f}")
    print(f"VPult: {calculate_VPult(fdn)[0]:.2f}")
    print(f"fcv: {calculate_fcv(fdn)[0]:.2f}")
    print(f"fVP: {calculate_fVP(fdn)[0]:.2f}")
    print(f"dvL: {calculate_dvL(fdn)[0]:.2f}")
    print(f"dvW: {calculate_dvW(fdn)[0]:.2f}")
    print(f"VOultL: {calculate_VOultL(fdn)[0]:.2f}")
    print(f"VOultW: {calculate_VOultW(fdn)[0]:.2f}")
    print(f"MOultL: {calculate_MOultL(fdn)[0]:.2f}")
    print(f"MOultW: {calculate_MOultW(fdn)[0]:.2f}")
    print(f"ex1L: {calculate_ex1L(fdn)[0]:.6f}")
    print(f"ex1W: {calculate_ex1W(fdn)[0]:.6f}")
    print(f"kvL: {calculate_kvL(fdn)[0]:.2f}")
    print(f"kvW: {calculate_kvW(fdn)[0]:.2f}")
    print(f"AngleL: {calculate_AngleL(fdn)[0]:.2f}")
    print(f"AngleW: {calculate_AngleW(fdn)[0]:.2f}")
    print(f"ks: {calculate_ks(fdn)[0]:.2f}")
    print(f"fVucL: {calculate_fVucL(fdn)[0]:.2f}")
    print(f"fVucW: {calculate_fVucW(fdn)[0]:.2f}")
    print(f"Bpr: {calculate_Bpr(fdn)[0]:.2f}")
    print(f"Mur: {calculate_Mur(fdn)[0]:.2f}")
    print(f"VPr: {calculate_VPr(fdn)[0]:.2f}")
    print(f"VOr: {calculate_VOr(fdn)[0]:.2f}")
    print(f"Cost: {calculate_Cost(fdn)[0]:.2f}")


In [144]:
test = Foundation(L=6.5,W=6.2,D=2.5,ColL=0.8,ColW=0.5,Pdl=8000,Pll=1500,BP=250,fc=25,barL=0.040,ctsL=0.1,barW=0.040,ctsW=0.1)
print_foundation_results(test)

Results for L=6.500, W=6.200, D=2.500, barL=0.040, ctsL=0.100, barW=0.040, ctsW=0.100
SWt: 604.50
Pult: 12575.40
BPmax: 250.73
BPult: 312.04
AstL: 12566.37
AstW: 12566.37
dsL: 2.48
dsW: 2.44
AstminL: 2872.98
AstminW: 2920.08
alpha: 0.81
gamma: 0.91
MultL: 1316.72
MultW: 1277.12
AstshrL: 1269.19
AstshrW: 1251.20
AstreqL: 2872.98
AstreqW: 2920.08
kuL: 0.00
kuW: 0.01
phiL: 0.85
phiW: 0.85
fMuoL: 12418.94
fMuoW: 12205.32
CLR: 3050.05
VPult: 9525.35
fcv: 1.91
fVP: 40374.56
dvL: 2.23
dvW: 2.20
VOultL: 195.25
VOultW: 195.25
MOultL: 63.85
MOultW: 63.85
ex1L: 0.000078
ex1W: 0.000078
kvL: 0.14
kvW: 0.15
AngleL: 29.54
AngleW: 29.54
ks: 0.50
fVucL: 562.85
fVucW: 560.01
Bpr: 1.00
Mur: 0.11
VPr: 0.24
VOr: 0.35
Cost: 75562.36


In [145]:
params = Foundation(ColL=0.8,ColW=0.5,Pdl=8000,Pll=1500,BP=250,fc=65,barL=0.040,ctsL=0.1,barW=0.040,ctsW=0.1)
params.L = calc_min_L(params)
params.W = params.L+params.ColW-params.ColL
temparray = calc_D_range(params)
del(params)
startingfdn = Foundation(L=temparray[0],W=temparray[1],D=temparray[2],Cvr=temparray[3],ColL=temparray[4],ColW=temparray[5],Pdl=temparray[6],Pll=temparray[7],BP=temparray[8],fc=temparray[9],barL=temparray[10],ctsL=temparray[11],barW=temparray[12],ctsW=temparray[13])
mask = calculate_fVucL(startingfdn)/calculate_VOultL(startingfdn) > 1
print(np.min(startingfdn.D[mask]))
del(startingfdn)
params = Foundation(ColL=0.8,ColW=0.5,Pdl=8000,Pll=1500,BP=250,fc=25,barL=0.012,ctsL=0.3,barW=0.012,ctsW=0.3)
params.L = calc_min_L(params)
params.W = params.L+params.ColW-params.ColL
temparray = calc_D_range(params)
del(params)
startingfdn = Foundation(L=temparray[0],W=temparray[1],D=temparray[2],Cvr=temparray[3],ColL=temparray[4],ColW=temparray[5],Pdl=temparray[6],Pll=temparray[7],BP=temparray[8],fc=temparray[9],barL=temparray[10],ctsL=temparray[11],barW=temparray[12],ctsW=temparray[13])
mask = calculate_fVucL(startingfdn)/calculate_VOultL(startingfdn) > 1
print(startingfdn.W[32])
print(calculate_SWt(startingfdn)[32])
print(calculate_VOultL(startingfdn)[32])
print(np.min(startingfdn.D[mask]))
del(startingfdn)


1.1999999999999997
6.000000000000001
578.3399999999998
149.88026831746086
2.549999999999998
